## Importing of Libraries


In [2]:
import time
from copy import deepcopy

import numpy as np
import torch
from dlordinal.datasets import FGNet
from dlordinal.models import OBDECOCModel
from dlordinal.losses import OrdinalECOCDistanceLoss
from sklearn.metrics import (accuracy_score, cohen_kappa_score,
                             confusion_matrix, mean_absolute_error)
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils import class_weight
from torch import cuda
from torch.optim import Adam
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, ToTensor
from torchvision.models import resnet18
from tqdm import tqdm

## Load and preprocess of FGNet dataset

First, we present the configuration parameters for the experimentation and the number of workers for the *DataLoader*, which defines the number of subprocesses to use for data loading. In this specific case, it refers to the images.

In [3]:
optimiser_params = {
    'lr': 1e-3,
    'bs': 200,
    'epochs': 5,
    's': 2,
    'c': 0.2,
    'beta': 0.5
}

workers = 3

Now we use the *FGNet* method to download and preprocess the images. Once that is done with the training data, we create a validation partition comprising 15% of the data using the *StratifiedShuffleSplit* method. Finally, with all the partitions, we load the images using a method called *DataLoader*.

In [7]:
fgnet = FGNet(root="./datasets/fgnet", download=True, process_data=True)

complete_train_data = ImageFolder(
    root="./datasets/fgnet/FGNET/train", transform=Compose([ToTensor()])
)
test_data = ImageFolder(
    root="./datasets/fgnet/FGNET/test", transform=Compose([ToTensor()])
)

num_classes = len(complete_train_data.classes)
classes = complete_train_data.classes
targets = complete_train_data.targets

# Create a validation split
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=0)
sss_splits = list(
    sss.split(X=np.zeros(len(complete_train_data)), y=complete_train_data.targets)
)
train_idx, val_idx = sss_splits[0]

# Create subsets for training and validation
train_data = Subset(complete_train_data, train_idx)
val_data = Subset(complete_train_data, val_idx)

# Get CUDA device
device = "cuda" if cuda.is_available() else "cpu"
print(f"Using {device} device")

# Create dataloaders
train_dataloader = DataLoader(
    train_data, batch_size=optimiser_params["bs"], shuffle=True, num_workers=workers
)
val_dataloader = DataLoader(
    val_data, batch_size=optimiser_params["bs"], shuffle=True, num_workers=workers
)
test_dataloader = DataLoader(
    test_data, batch_size=optimiser_params["bs"], shuffle=False, num_workers=workers
)

# Get image shape
img_shape = None
for X, _ in train_dataloader:
    img_shape = list(X.shape[1:])
    break
print(f"Detected image shape: {img_shape}")

# Define class weights for imbalanced datasets
class_weights = torch.from_numpy(class_weight.compute_class_weight(
    "balanced", classes=[int(c) for c in classes], y=targets
)).float().to(device)
print(f"{class_weights=}")

Files already downloaded and verified
Files already processed and verified
Files already split and verified
Using cpu device
Detected image shape: [3, 128, 128]
class_weights=tensor([1.5523, 0.5256, 1.1033, 0.8241, 1.0680, 2.5189])


## Model and optimiser

We are using a modified version of the *ResNet* architecture, specifically designed for the loss function explained in the next section. In this case, the *ResNet* model is not pretrained with *ImageNet*, so we will need to undergo an extensive learning process.

To adapt the outputs of the model to this, the final fully-connected block is substituted by $Q-1$ fully-connected blocks [1], each one with a single output unit with sigmoid activation.

As an alternative to the *ResNet* architecture, you can use the *VGG* architecture, which has also been implemented to work with the loss function explained in the following sections.

Finally, we define the *Adam* optimiser, which is used to adjust the network's weights and minimize the error of a loss function.

[1]: Barbero-Gómez, J., Gutiérrez, P. A., & Hervás-Martínez, C. (2022). *Error-correcting output codes in the framework of deep ordinal classification.* Neural Processing Letters, 1-32. Springer.

In [8]:
model = OBDECOCModel(num_classes, resnet18(num_classes=1000), base_n_outputs=1000).to(device)

# Optimizer
optimizer = Adam(model.parameters(), lr=optimiser_params['lr'])

## Loss Function

The original $Q$-class ordinal problem is decomposed into $Q-1$ binary decision problems, what is known as Ordinal Binary Decomposition (ODB) [1]. So the categorical cross-entropy has been substituted by the Mean Squared Error loss because it copes better with the distance function used for the Error-Conecting Output Codes (ECOC) decision:

$$
\ell = \frac{1}{N} ∑_{i=1}^N ∑_{k=1}^{Q-1} (\mathbf{1} \{y_i \succ \mathcal{C}_k\} - P(y_i \succ \mathcal{C}_k | x_i))^2
$$

where $\mathbf{1} \{y_i \succ \mathcal{C}_k\}$ is the indicator function that is equal to 1 when $y_i \succ \mathcal{C}_k$ and 0 otherwise, and $P(y_i \succ \mathcal{C}_k | x_i)$ is the probability that $y_i \succ \mathcal{C}_k$ predicted by the network given a sample.

[1]: Barbero-Gómez, J., Gutiérrez, P. A., & Hervás-Martínez, C. (2022). *Error-correcting output codes in the framework of deep ordinal classification.* Neural Processing Letters, 1-32. Springer. doi.org/10.1007/s11063-022-10824-7

In [9]:
loss_fn = OrdinalECOCDistanceLoss(
    num_classes=num_classes, weights=class_weights
).to(device)

## Metrics

In [10]:
# Metrics computation


def compute_metrics(y_true: np.ndarray, 
    y_pred: np.ndarray, 
    num_classes: int):

    if len(y_true.shape) > 1:
        y_true = np.argmax(y_true, axis=1)

    if len(y_pred.shape) > 1:
        y_pred = np.argmax(y_pred, axis=1)

    labels = range(0, num_classes)

    # Metrics calculation
    qwk = cohen_kappa_score(y_true, y_pred, weights='quadratic', labels=labels)
    ms = minimum_sensitivity(y_true, y_pred, labels=labels)
    mae = mean_absolute_error(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    off1 = accuracy_off1(y_true, y_pred, labels=labels)
    conf_mat = confusion_matrix(y_true, y_pred, labels=labels)

    metrics = {
        'QWK': qwk,
        'MS': ms,
        'MAE': mae,
        'CCR': acc,
        '1-off': off1,
        'Confusion matrix': conf_mat
    }

    return metrics


def _compute_sensitivities(y_true, y_pred, labels=None):
	if len(y_true.shape) > 1:
		y_true = np.argmax(y_true, axis=1)
	if len(y_pred.shape) > 1:
		y_pred = np.argmax(y_pred, axis=1)

	conf_mat = confusion_matrix(y_true, y_pred, labels=labels)

	sum = np.sum(conf_mat, axis=1)
	mask = np.eye(conf_mat.shape[0], conf_mat.shape[1])
	correct = np.sum(conf_mat * mask, axis=1)
	sensitivities = correct / sum

	sensitivities = sensitivities[~np.isnan(sensitivities)]

	return sensitivities


def minimum_sensitivity(y_true, y_pred, labels=None):
	return np.min(_compute_sensitivities(y_true, y_pred, labels=labels))


def accuracy_off1(y_true, y_pred, labels=None):
	if len(y_true.shape) > 1:
		y_true = np.argmax(y_true, axis=1)
	if len(y_pred.shape) > 1:
		y_pred = np.argmax(y_pred, axis=1)

	conf_mat = confusion_matrix(y_true, y_pred, labels=labels)
	n = conf_mat.shape[0]
	mask = np.eye(n, n) + np.eye(n, n, k=1), + np.eye(n, n, k=-1)
	correct = mask * conf_mat

	return 1.0 * np.sum(correct) / np.sum(conf_mat)


def print_metrics(metrics):
    print("")
    print('Confusion matrix :\n{}'.format(metrics['Confusion matrix']))
    print("")
    print('MS: {:.4f}'.format(metrics['MS']))
    print("")
    print('QWK: {:.4f}'.format(metrics['QWK']))
    print("")
    print('MAE: {:.4f}'.format(metrics['MAE']))
    print("")
    print('CCR: {:.4f}'.format(metrics['CCR']))
    print("")
    print('1-off: {:.4f}'.format(metrics['1-off']))

## Training process

In [11]:
def train(
    dataloader: torch.utils.data.DataLoader,
    model: OBDECOCModel,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    device: torch.device,
    H: dict,
    num_classes: int,
):  # H: dict
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    progress_bar = tqdm(total=num_batches, ncols=100, position=0, desc="Train progress")
    model.train()
    mean_loss, accuracy = 0, 0
    y_pred, y_true = None, None

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)  # Inputs and labels to device

        # Compute prediction error and accuracy of the training process
        pred = model(X)
        print(pred)
        print(y)
        loss = loss_fn(pred, y)

        mean_loss += loss
        accuracy += (pred.argmax(1) == y).type(torch.float).sum().item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Stack predictions and true labels to determine the confusion matrix
        pred_np = model.transformer.labels(pred).cpu().detach().numpy()
        true_np = y.cpu().detach().numpy()
        if y_pred is None:
            y_pred = pred_np
        else:
            y_pred = np.concatenate((y_pred, pred_np))

        if y_true is None:
            y_true = true_np
        else:
            y_true = np.concatenate((y_true, true_np))

        # Update progress bar
        progress_bar.set_postfix(loss=loss.item(), accuracy=accuracy)
        progress_bar.update(1)

    accuracy /= size
    mean_loss /= num_batches

    H["train_loss"].append(loss.cpu().detach().numpy())
    H["train_acc"].append(accuracy)

    # Confusion matrix for training
    labels = range(0, num_classes)
    conf_mat = confusion_matrix(y_true, y_pred, labels=labels)
    print("")
    print("Train Confusion matrix :\n{}".format(conf_mat))
    print("")

    return accuracy, mean_loss

## Testing process

In [12]:
def test(
    test_dataloader: torch.utils.data.DataLoader,
    model: OBDECOCModel,
    loss_fn: torch.nn.Module,
    device: torch.device,
    num_classes: int,
):
    num_batches = len(test_dataloader)
    progress_bar = tqdm(total=num_batches, ncols=100, position=0, desc="Test progress")
    model.eval()
    test_loss = 0
    y_pred, y_true = None, None

    with torch.no_grad():
        for batch, (X, y) in enumerate(test_dataloader):
            X, y = X.to(device), y.to(device)  # inputs and labels to device
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

            # Stack predictions and true labels
            pred_np = model.transformer.labels(pred).cpu().detach().numpy()
            true_np = y.cpu().detach().numpy()
            if y_pred is None:
                y_pred = pred_np
            else:
                y_pred = np.concatenate((y_pred, pred_np))

            if y_true is None:
                y_true = true_np
            else:
                y_true = np.concatenate((y_true, true_np))

            # Update progress bar
            progress_bar.set_postfix(loss=test_loss / (batch + 1))
            progress_bar.update(1)

    test_loss /= num_batches
    metrics = compute_metrics(y_true, y_pred, num_classes)
    print_metrics(metrics)

    return metrics, test_loss

## Validation process

In [13]:
def validate(
    dataloader: torch.utils.data.DataLoader,
    model: OBDECOCModel,
    loss_fn: torch.nn.Module,
    device: torch.device,
    H: dict,
    num_classes: int,
):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    loss, accuracy = 0, 0
    y_pred, y_true = None, None

    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss += loss_fn(pred, y)
            accuracy += (pred.argmax(1) == y).type(torch.float).sum().item()

            pred_np = model.transformer.labels(pred).cpu().detach().numpy()
            true_np = y.cpu().detach().numpy()
            if y_pred is None:
                y_pred = pred_np
            else:
                y_pred = np.concatenate((y_pred, pred_np))

            if y_true is None:
                y_true = true_np
            else:
                y_true = np.concatenate((y_true, true_np))

    accuracy /= size
    loss /= num_batches

    H["val_loss"].append(loss.cpu().detach().numpy())
    H["val_acc"].append(accuracy)

    metrics = compute_metrics(y_true, y_pred, num_classes)

    return metrics, accuracy, loss

## Results

In [14]:
H = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

# To store validation metrics
validation_metrics = {}

# Definition to store best model weights
best_model_weights = model.state_dict()
best_qwk = 0.0

# Start time
start_time = time.time()

for e in range(optimiser_params["epochs"]):
    train_acc, train_loss = train(
        train_dataloader, model, loss_fn, optimizer, device, H, num_classes=num_classes
    )
    validation_metrics, val_acc, val_loss = validate(
        val_dataloader, model, loss_fn, device, H, num_classes=num_classes
    )

    if validation_metrics["QWK"] >= best_qwk:
        best_qwk = validation_metrics["QWK"]
        best_model_weights = deepcopy(model.state_dict())

    print("[INFO] EPOCH: {}/{}".format(e + 1, optimiser_params["epochs"]))
    print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(train_loss, train_acc))
    print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(val_loss, val_acc))

# Store last train error
train_error = H["train_loss"][-1]

# Restore best weights
model.load_state_dict(best_model_weights)

# Start evaluation
print("[INFO] Network evaluation ...")

test_metrics, test_loss = test(
    test_dataloader, model, loss_fn, device, num_classes=num_classes
)

# End time
end_time = time.time()
print("\n[INFO] Total training time: {:.2f}s".format(end_time - start_time))

Train progress:   0%|                                                         | 0/4 [00:00<?, ?it/s]

tensor([[0.4912, 0.5440, 0.5716, 0.5553, 0.4533],
        [0.4801, 0.5443, 0.5829, 0.6065, 0.4515],
        [0.4903, 0.5389, 0.5636, 0.5578, 0.4593],
        [0.4928, 0.5433, 0.5640, 0.5893, 0.4636],
        [0.4999, 0.5485, 0.5699, 0.5603, 0.4691],
        [0.4862, 0.5494, 0.5668, 0.5821, 0.4698],
        [0.4791, 0.5328, 0.5517, 0.5645, 0.4729],
        [0.4375, 0.6252, 0.7488, 0.7413, 0.3644],
        [0.4941, 0.5313, 0.5622, 0.5524, 0.4552],
        [0.4983, 0.5349, 0.5862, 0.5830, 0.4612],
        [0.4672, 0.5675, 0.6398, 0.6316, 0.4874],
        [0.4821, 0.5367, 0.5761, 0.5522, 0.4565],
        [0.4852, 0.5478, 0.5606, 0.5579, 0.4540],
        [0.4795, 0.5650, 0.5509, 0.5663, 0.4736],
        [0.4718, 0.5532, 0.5644, 0.5718, 0.4818],
        [0.4679, 0.5675, 0.5650, 0.5655, 0.4665],
        [0.4932, 0.5563, 0.6577, 0.6633, 0.4699],
        [0.5057, 0.5309, 0.5646, 0.5648, 0.4514],
        [0.4898, 0.5299, 0.5731, 0.5509, 0.4546],
        [0.4771, 0.5542, 0.5730, 0.5579, 0.4536],


Train progress:  25%|██████▌                   | 1/4 [00:04<00:13,  4.65s/it, accuracy=39, loss=247]

tensor([[0.9139, 0.7853, 0.5700, 0.3353, 0.1514],
        [0.9003, 0.7873, 0.5944, 0.3545, 0.1293],
        [0.9694, 0.8326, 0.1579, 0.0480, 0.0520],
        [0.9099, 0.7803, 0.4800, 0.2738, 0.1233],
        [0.9306, 0.8188, 0.5699, 0.3504, 0.1370],
        [0.9394, 0.7702, 0.3242, 0.1758, 0.1025],
        [0.9107, 0.7885, 0.4797, 0.2544, 0.1268],
        [0.9571, 0.8189, 0.2473, 0.0989, 0.0865],
        [0.9079, 0.7911, 0.5795, 0.3759, 0.1344],
        [0.9065, 0.7888, 0.6060, 0.3557, 0.1395],
        [0.9172, 0.8066, 0.6392, 0.3711, 0.1283],
        [0.9075, 0.7870, 0.5617, 0.3215, 0.1370],
        [0.9215, 0.8045, 0.5229, 0.3068, 0.1261],
        [0.9176, 0.7832, 0.4543, 0.2241, 0.1156],
        [0.9156, 0.7820, 0.5722, 0.3071, 0.1388],
        [0.9058, 0.7824, 0.5835, 0.3727, 0.1328],
        [0.9089, 0.7883, 0.5883, 0.3744, 0.1441],
        [0.9253, 0.8015, 0.5002, 0.2497, 0.1266],
        [0.8983, 0.7731, 0.5641, 0.3209, 0.1185],
        [0.9250, 0.7745, 0.3828, 0.1647, 0.1129],


Train progress:  50%|█████████████             | 2/4 [00:09<00:09,  4.70s/it, accuracy=67, loss=199]

...

Train progress: 100%|█████████████████████████| 4/4 [00:14<00:00,  3.55s/it, accuracy=73, loss=35.5]


Train Confusion matrix :
[[46 25  1  1  0  0]
 [56 84 39 25 11  1]
 [ 5 21 33 33  9  2]
 [ 6 11 26 55 32  7]
 [ 0  0  2 38 48 18]
 [ 0  0  0  9 14 22]]



[INFO] EPOCH: 5/5
Train loss: 82.521561, Train accuracy: 0.1074
Val loss: 151.584747, Val accuracy: 0.2975

[INFO] Network evaluation ...


Test progress: 100%|████████████████████████████████████████| 2/2 [00:02<00:00,  1.05s/it, loss=135]


Confusion matrix :
[[17  0  2  4  0  0]
 [39  0  4  5  0  0]
 [29  0  7  6  1  0]
 [30  0 12  6  0  0]
 [14  0  0  9  0  0]
 [ 6  0  3  7  0  0]]

MS: 0.0000

QWK: 0.1455

MAE: 1.7662

CCR: 0.1493

1-off: 0.4975

[INFO] Total training time: 85.39s
